In [4]:
#!git clone https://github.com/mymusise/ChatGLM-Tuning.git
%cd  /home/wanddy/ChatGLM-Tuning/
!pip install -r requirements.txt 

/home/wanddy/ChatGLM-Tuning
Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
  Cloning https://ghproxy.com/github.com/huggingface/peft.git to /tmp/pip-req-build-s_apeqeh
  Running command git clone --filter=blob:none --quiet https://ghproxy.com/github.com/huggingface/peft.git /tmp/pip-req-build-s_apeqeh
  Resolved https://ghproxy.com/github.com/huggingface/peft.git to commit 098962fa6515f2e4fe83a757f5995d3ffbb1c373
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/ec/18/75dbd7529844c8600944df123160216323982d39d24a30e9f6806279f935/bitsandbytes-0.37.1-py3-none-any.whl (76.3 MB)
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/6d/9b/2f536f9e73390209e0b27b74691355dac494b7ec8154f3012fdc6debbae7/transformers-4.27.1-py3-none-any.whl (6.7 MB)
  Created wheel for peft: filename=peft-0.3.0.dev0-py3-none-any.whl size=40691 sha256

In [5]:
!python cover_alpaca2jsonl.py \
    --data_path data/alpaca_data.json \
    --save_path data/alpaca_data.jsonl

formatting..: 100%|███████████████████| 52002/52002 [00:00<00:00, 185634.83it/s]


In [6]:
!python tokenize_dataset_rows.py \
    --jsonl_path data/alpaca_data.jsonl \
    --save_path data/alpaca \
    --max_seq_length 128

Generating train split: 0 examples [00:00, ? examples/s]Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.

Generating train split: 1 examples [04:13, 253.62s/ examples]02 [00:00<?, ?it/s]
Generating train split: 169 examples [04:13,  1.05s/ examples]0:31, 1654.03it/s]
Generating train split: 351 examples [04:13,  2.41 examples/s]0:29, 1743.15it/s]
Generating train split: 529 examples [04:13,  4.45 examples/s]0:29, 1751.50it/s]
Generating train split: 707 examples [04:14,  7.35 examples/s]0:28, 1769.23it/s]
Generating train split: 921 examples [04:14, 12.31 examples/s]0:32, 1574.38it/s]
Generating train split: 1099 examples [04:14, 18.15 examples/s]:33, 1519.54it/s]
Generating train split: 1298 examples [04:14, 27.44 examples

In [1]:
import sys

sys.path.append("../")

import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [3]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


model = ChatGLMForConditionalGeneration.from_pretrained("THUDM/chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
Loading checkpoint shards: 100%|██████████| 8/8 [00:11<00:00,  1.50s/it]


In [4]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


## Test before finetune

In [6]:
%cd  /home/wanddy/ChatGLM-Tuning/
from cover_alpaca2jsonl import format_example
import json


instructions = json.load(open("data/alpaca_data.json"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/home/wanddy/ChatGLM-Tuning


/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruction: Give three tips for staying healthy.
Answer: I'm sorry, but I'm not sure what you're asking. Could you please provide more context or clarify your question?
### 1.Answer:
 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule. 


Instruction: What are the three primary colors?
Answer: The three primary colors in painting are red, blue, and green. These colors are used to create a range of other colors, including yellow, orange, and pink. The use of these colors in painting is a key aspect of the art form.
### 2.Answer:
 The three primary colors are red, blue, and yellow. 


Instruction: Describe the structure of an atom.
Answer: The原子是构成物质的基本单位,由带正电荷的质子、带负电荷的电子和中间由电子围绕的原子核组成。原子核由质子和中子组成,中子在原子核内部运动,质子则围绕原子核旋转。

原子通过化学反应与其他物质发生相互作用。化学反应是由原子之间的相互作用引起的,因此了解原子的结构和相互作用是理解化学和物质世界的基础。

原子的大小通常用其半径来表示,半径越小,原子越紧密,其化学性质也越不稳定。原子的大小取决于原子的质子

In [7]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True

In [8]:
import datasets

dataset_path = "data/alpaca/"

dataset = datasets.load_from_disk(dataset_path)

train_num = 500

mini_train_dataset = datasets.Dataset.from_dict(dataset[:train_num])

In [10]:
from transformers import Trainer, HfArgumentParser


def get_masks_and_position_ids(
    seq, seq_len, context_length, device, gmask=False, position_encoding_2d=True
):
    mask_position = (
        seq_len - 2
    )  # is equal to `seq.index(mask_token)` or `seq.index(150001)`
    attention_mask = torch.ones((1, context_length, context_length), device=device)
    attention_mask.tril_()
    attention_mask[..., : mask_position - 1] = 1
    attention_mask = (attention_mask < 0.5).bool()

    if position_encoding_2d:
        seq_length = seq_len - 1  # is equal to `seq_length = seq.index(150004)`
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[seq_length:] = mask_position
        block_position_ids = torch.cat(
            (
                torch.zeros(seq_length, dtype=torch.long, device=device),
                torch.arange(
                    context_length - seq_length, dtype=torch.long, device=device
                )
                + 1,
            )
        )
        position_ids = torch.stack((position_ids, block_position_ids), dim=0)
    else:
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[context_length - 1 :] = mask_position
    return attention_mask, position_ids


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    attention_mask_list = []
    position_ids_list = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1)
            + ids[(seq_len - 1) :]
            + [-100] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        attention_mask, position_ids = get_masks_and_position_ids(
            ids, seq_len, longest, _ids.device, gmask=False
        )
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
        attention_mask_list.append(attention_mask)
        position_ids_list.append(position_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    attention_mask = torch.stack(attention_mask_list)
    position_ids = torch.stack(position_ids_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask,
        "position_ids": position_ids,
    }


class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            position_ids=inputs["position_ids"],
            labels=inputs["labels"],
        ).loss
    
    def save_model(self, output_dir=None, _internal_call=False):
        from transformers.trainer import TRAINING_ARGS_NAME
        os.makedirs(output_dir, exist_ok=True)
        torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))
        saved_params = {
            k: v.to("cpu") for k, v in self.model.named_parameters() if v.requires_grad
        }
        torch.save(saved_params, os.path.join(output_dir, "adapter_model.bin"))

In [11]:
training_args = TrainingArguments(
    "output",
    fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,
    learning_rate = 1e-4,
    max_steps=1500,
    logging_steps=50,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)
trainer.train()
# save model
model.save_pretrained(training_args.output_dir)

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/1500 [00:00<?, ?it/s]/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
  3%|▎         | 50/1500 [00:56<24:55,  1.03s/it] 

{'loss': 2.9924, 'learning_rate': 9.713333333333334e-05, 'epoch': 0.1}


  7%|▋         | 100/1500 [01:51<27:17,  1.17s/it]

{'loss': 2.2892, 'learning_rate': 9.38e-05, 'epoch': 0.2}


 10%|█         | 150/1500 [02:46<24:34,  1.09s/it]

...

100%|██████████| 1500/1500 [28:19<00:00,  1.10s/it]

{'loss': 1.3097, 'learning_rate': 4.666666666666667e-07, 'epoch': 3.0}


100%|██████████| 1500/1500 [28:31<00:00,  1.14s/it]

{'train_runtime': 1711.9855, 'train_samples_per_second': 0.876, 'train_steps_per_second': 0.876, 'train_loss': 1.5900354690551757, 'epoch': 3.0}


TrainOutput(global_step=1500, training_loss=1.5900354690551757, metrics={'train_runtime': 1711.9855, 'train_samples_per_second': 0.876, 'train_steps_per_second': 0.876, 'train_loss': 1.5900354690551757, 'epoch': 3.0})

## Test After finetune

In [12]:
from cover_alpaca2jsonl import format_example
import json


instructions = json.load(open("data/alpaca_data.json"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruction: Give three tips for staying healthy.
Answer: 1. Eat a balanced diet.
2. Get regular exercise.
3. Stay hydrated.
### 1.Answer:
 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule. 


Instruction: What are the three primary colors?
Answer: The three primary colors are red, blue, and yellow.
### 2.Answer:
 The three primary colors are red, blue, and yellow. 


Instruction: Describe the structure of an atom.
Answer: An atom is made up of a small number of particles called electrons, protons, and neutrons. Electrons are the most important part of an atom, as they are responsible for the properties of an atom. Protons and neutrons are also important, as they are responsible for the properties of an atom. Electrons are responsible for the properties of an atom, such as its charge and its ability to attract or repel each other. Prot

In [13]:
import os


def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)


save_tunable_parameters(model, os.path.join("output", "chatglm-lora.pt"))